## Train regression models for demand forecasting

This script trains regression models using 
- **Weather data** (past & future)
- **Demand data** (past observations)

to predict demand based on:
- **Weather data** (future)

Models to consider: MLP, linear regression, polynomial regression (4th degree from Dessler’s talk), long short-term memory, Random Forest, Support Vector Regression

### Import Packages

In [7]:
import numpy as np
import pyarrow.parquet as pq
import joblib
import scipy
import pandas as pd
import time
import os
import re
import glob
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import sklearn
from sklearn.model_selection import TimeSeriesSplit
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.neural_network import MLPRegressor as MLP
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from datetime import datetime, timedelta
import seaborn as sns
import pytz
import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)
import yaml
import pprint
import pwlf

from src import input_ops
from src import model_ops

### Load config file with scenarios and parameters 

In [8]:
config_file_name = 'config1'; config_path = f"config/{config_file_name}.yaml"; config = input_ops.load_config(config_path)

aggregation_level = config['aggregation_level']
smart_ds_year = config['smart_ds_years'][0]
building_types = config["building_types"]
input_data_dict_name = config['input_data_dict_name']
Y_column = config['Y_column']
X_columns = config['X_columns']

## Initialize parameters for saving paths
output_path_str = config['output_data_training_path']
aggregation_level = config['aggregation_level']

if Y_column == 'cooling_kw_sum':
    prediction_model_str = config['prediction_model_cooling']
elif Y_column == 'heating_kw':
    prediction_model_str = config['prediction_model_heating']
else:
    raise ValueError('Model architecture is not defined for model output! (only for cooling_sum and heating)')
    
# pprint.pprint(config, sort_dicts=False) # print config
print(f"Aggregation level: {config['aggregation_level']} \n SMART-DS year: {config['smart_ds_years'][0]} \n Months: {config['start_month']}-{config['end_month']} \n output: {config['Y_column']}, \n inputs: {config['X_columns_set']} \n Prediction model: {prediction_model_str} \n output directory:{config['output_data_training_path']}")

Aggregation level: feeder 
 SMART-DS year: 2018 
 Months: 1-12 
 output: cooling_kw_sum, 
 inputs: X_columns_D 
 Prediction model: MLP_2_512_64 
 output directory:/nfs/turbo/seas-mtcraig-climate/Aviad/load_prediction/results/data/training/2018/months_1_12/ml_output_data/cooling_kw_sum/X_columns_D/feeder/


### Load input data (Resstock weather data & smart-ds load data)

In [3]:
## Load input load & weather data of year-city-region-building_type combinations
loaded_input_data_dict = {}
if config['aggregation_level'] == 'building': # Assuming buildings have separate joblib files
    for city, region in CITY_REGIONS_TO_RUN.items():
        path_temp = os.path.join(config["input_data_training_path"], f"{city}/{region}/{input_data_dict_name}.joblib")
        print(path_temp)
        # loaded_input_data_dict[(smart_ds_year,city,region,load_model,building_types)] = joblib.load(os.path.join(config["input_data_training_path"], f"{city}/{region}/{input_data_dict_name}.joblib"))
else:
    print(f'Loading dictionary with load & weather data from {config["input_data_training_path"]}')
    loaded_input_data_dict = joblib.load(os.path.join(config["input_data_training_path"], f"{input_data_dict_name}.joblib")) # Load dictionary with load & weather data 

Loading dictionary with load & weather data from /nfs/turbo/seas-mtcraig-climate/Aviad/load_prediction/results/data/training/2018/months_1_12/ml_input_data/resstock/amy2018/feeder/


### Train ML models and save models, scalers and metrics
---

In [4]:
## Train MLP model for year-city-region-building_type combinations
start_time = time.time()

## Initialize dictionaries
mlp_results = {} # change to mlp_metrics_dict
mlp_model_dict = {}
xnorm_dict = {}
ynorm_dict = {}

for smart_ds_year, city, region, load_model, building_type in loaded_input_data_dict.keys():
    print(f".......Training {prediction_model_str} for {smart_ds_year} {city} {region} {load_model} {building_type}.......")
    # Train MLP model

    # Load data
    if not config['aggregation_level'] == 'building':
        input_df = loaded_input_data_dict[(smart_ds_year, city, region, load_model, building_type)]
    else:
        print("To train model at the building level: (1) create processed input data at the building level, and (2) update code here to load it")

    match prediction_model_str:
        case 'MLP_2_1452_10':
            first_L_n = 1452
            second_L_n = 10
            MLP_model, xnorm, ynorm, y_test, y_test_pred, y_train, y_train_pred, training_runtime = model_ops.train_mlp_model(input_df, X_columns, Y_column, first_L_n, second_L_n)
        case 'MLP_2_4096_512': 
            first_L_n = 2048
            second_L_n = 256
            MLP_model, xnorm, ynorm, y_test, y_test_pred, y_train, y_train_pred, training_runtime = model_ops.train_mlp_model(input_df, X_columns, Y_column, first_L_n, second_L_n) 
        case 'MLP_2_2048_512': 
            first_L_n = 4096
            second_L_n = 512
            MLP_model, xnorm, ynorm, y_test, y_test_pred, y_train, y_train_pred, training_runtime = model_ops.train_mlp_model(input_df, X_columns, Y_column, first_L_n, second_L_n)    
        case 'MLP_2_1024_128': 
            first_L_n = 1024
            second_L_n = 128
            MLP_model, xnorm, ynorm, y_test, y_test_pred, y_train, y_train_pred, training_runtime = model_ops.train_mlp_model(input_df, X_columns, Y_column, first_L_n, second_L_n)
        case 'MLP_2_512_64': 
            first_L_n = 512
            second_L_n = 64
            MLP_model, xnorm, ynorm, y_test, y_test_pred, y_train, y_train_pred, training_runtime = model_ops.train_mlp_model(input_df, X_columns, Y_column, first_L_n, second_L_n)
        case 'MLP_2_256_32':
            first_L_n = 256
            second_L_n = 32
            MLP_model, xnorm, ynorm, y_test, y_test_pred, y_train, y_train_pred, training_runtime = model_ops.train_mlp_model(input_df, X_columns, Y_column, first_L_n, second_L_n)
        case 'MLP_2_100_10':
            first_L_n = 100
            second_L_n = 10
            MLP_model, xnorm, ynorm, y_test, y_test_pred, y_train, y_train_pred, training_runtime = model_ops.train_mlp_model(input_df, X_columns, Y_column, first_L_n, second_L_n)
        case 'MLP_2_32_16':
            first_L_n = 32
            second_L_n = 16
            MLP_model, xnorm, ynorm, y_test, y_test_pred, y_train, y_train_pred, training_runtime = model_ops.train_mlp_model(input_df, X_columns, Y_column, first_L_n, second_L_n)
        case 'MLP_1_16':
            first_L_n = 16
            second_L_n = ''
            MLP_model, xnorm, ynorm, y_test, y_test_pred, y_train, y_train_pred, training_runtime = model_ops.train_mlp_model(input_df, X_columns, Y_column, first_L_n, second_L_n)
        case 'PWLR_4bp':
            # X_columns = ['Dry Bulb Temperature [°C]']
            MLP_model, xnorm, ynorm, y_test, y_test_pred, training_runtime = model_ops.train_piecewise_linear_model(input_df, X_columns, Y_column, num_breakpoints=4)
        case 'Poly_4deg':
            MLP_model, poly_transform, xnorm, ynorm, y_test, y_test_pred, training_runtime = model_ops.train_polynomial_model(input_df, X_columns, Y_column, degree=4, alpha=1.0)
        case _:
            MLP_model, xnorm, ynorm, y_test, y_test_pred, training_runtime = model_ops.train_linear_model(input_df, X_columns, Y_column)

    # compute MLP performance metrics
    r2, mae, mape, rmse, nrmse, peak_load_error, peak_time_error, mape_high, nrmse_high = model_ops.metrics_test_data(y_test, y_test_pred)

    # Save MLP performance metrics
    mlp_results[(smart_ds_year, city, region, load_model, building_type)] = {
        "X_columns": X_columns,
        "Y_column": Y_column,
        "training_runtime": training_runtime,
        "y_test": y_test, 
        "y_test_pred": y_test_pred,
        "y_train": y_train, 
        "y_train_pred": y_train_pred,
        "r2": r2,
        "mae": mae,
        "mape": mape,
        "rmse": rmse,
        "nrmse": nrmse,
        "peak_load_error": peak_load_error,
        "peak_time_error": peak_time_error,
        "mape_high": mape_high,
        "nrmse_high": nrmse_high
    }
    
    # Save trained models and scalers in dictionaries
    mlp_model_dict[(smart_ds_year, city, region, load_model, building_type)] = MLP_model
    xnorm_dict[(smart_ds_year, city, region, load_model, building_type)] = xnorm
    ynorm_dict[(smart_ds_year, city, region, load_model, building_type)] = ynorm

# Save dictionaries as joblib files
joblib.dump(mlp_results, os.path.join(output_path_str, "metrics", f"{prediction_model_str}_results.joblib"))
joblib.dump(mlp_model_dict, os.path.join(output_path_str, "models", f"{prediction_model_str}_models_dict.joblib"))
joblib.dump(xnorm_dict, os.path.join(output_path_str, "scalers", f"{prediction_model_str}_xnorm_dict.joblib"))
joblib.dump(ynorm_dict, os.path.join(output_path_str, "scalers", f"{prediction_model_str}_ynorm_dict.joblib"))

print("All models, scalers, and results saved successfully!")

end_time = time.time(); print(f"Runtime for Training models: {(end_time - start_time) / 60:.2f} minutes")

.......Training MLP_1_16 for 2018 GSO rural rhs2_1247--rdt1262 res.......
.......Training MLP_1_16 for 2018 GSO rural rhs2_1247--rdt1262 com.......
.......Training MLP_1_16 for 2018 GSO rural rhs2_1247--rdt1264 res.......
.......Training MLP_1_16 for 2018 GSO rural rhs2_1247--rdt1264 com.......
.......Training MLP_1_16 for 2018 GSO rural rhs0_1247--rdt1527 res.......
.......Training MLP_1_16 for 2018 GSO rural rhs0_1247--rdt1527 com.......
.......Training MLP_1_16 for 2018 GSO rural rhs0_1247--rdt1948 res.......
.......Training MLP_1_16 for 2018 GSO rural rhs0_1247--rdt1948 com.......
.......Training MLP_1_16 for 2018 GSO rural rhs0_1247--rdt1534 res.......
.......Training MLP_1_16 for 2018 GSO rural rhs0_1247--rdt1534 com.......
.......Training MLP_1_16 for 2018 GSO rural rhs1_1247--rdt137 res.......
.......Training MLP_1_16 for 2018 GSO rural rhs1_1247--rdt137 com.......
.......Training MLP_1_16 for 2018 GSO rural rhs3_1247--rdt2999 com.......
.......Training MLP_1_16 for 2018 GSO ru

___

### Testing MLP-PSO

In [34]:
dataframe = loaded_input_data_dict[('2018', 'GSO', 'rural', 'regional', 'res')]
feature_columns = config['X_columns_D']
target_column = config['Y_column']

import numpy as np
import pandas as pd
import time
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor as MLP
from sklearn.metrics import r2_score
import pyswarms as ps


def stratified_temperature_split(X, y, num_bins, test_size, random_state):
    """
    Perform stratified sampling based on temperature bins to ensure 
    that the temperature distribution in the training and test sets is consistent.
    """
    if not isinstance(X.index, pd.DatetimeIndex):
        raise ValueError("The DataFrame index must be a DatetimeIndex")
    
    temperature = X["Dry Bulb Temperature [°C]"]
    X["temp_bin"] = pd.qcut(temperature, q=num_bins, labels=False, duplicates="drop")
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, stratify=X["temp_bin"], random_state=random_state
    )
    
    X_train = X_train.drop(columns=["temp_bin"])
    X_test = X_test.drop(columns=["temp_bin"])
    
    return X_train, X_test, y_train, y_test


def train_mlp_model(X_train, y_train, X_test, y_test, layers, alpha):
    """
    Train an MLP model with the given architecture.
    """
    xnorm = MinMaxScaler().fit(X_train)
    X_train_norm = xnorm.transform(X_train)
    X_test_norm = xnorm.transform(X_test)
    
    ynorm = MinMaxScaler().fit(y_train)
    y_train_norm = ynorm.transform(y_train)
    
    mlp_model = MLP(activation='relu', hidden_layer_sizes=tuple(layers), learning_rate='constant', max_iter=500,
                     solver='adam', random_state=42)
    mlp_model.fit(X_train_norm, y_train_norm)
    
    y_test_pred_norm = mlp_model.predict(X_test_norm).reshape(-1, 1)
    y_test_pred = ynorm.inverse_transform(y_test_pred_norm)
    
    # Compute R2 score
    r2 = r2_score(y_test, y_test_pred)
    
    # Compute peak-related metrics
    max_y_test = np.max(y_test)  # Max actual value
    max_model_pred = np.max(y_test_pred)  # Max predicted value
    
    # Ratio difference between max predicted and max actual
    peak_load_error = ((max_y_test - max_model_pred) / max_y_test)
    
    # Compute objective function
    objective_value = alpha * r2 + (1 - alpha) * peak_load_error
    
    return objective_value


def objective_function(solutions, X_train, y_train, X_test, y_test, alpha):
    """
    Objective function for optimization, minimizing the linear combination of R2 and peak_load_error.
    """
    costs = []
    for solution in solutions:
        layers = [max(1, int(round(neurons))) for neurons in solution]
        if not layers:
            costs.append(float("inf"))  # Prevent empty layers
    
        else:
            costs.append(train_mlp_model(X_train, y_train, X_test, y_test, layers, alpha))
    return np.array(costs)


def optimize_mlp_architecture(X_train, y_train, X_test, y_test, alpha, num_agents=10, num_iterations=50):
    """
    Optimize MLP architecture using a PSOGWO hybrid approach.
    """
    def fitness_function(solution):
        return objective_function(solution, X_train, y_train, X_test, y_test, alpha)
    
    lb = [5] * 3  # Lower bound: at least 5 neurons per layer
    ub = [100] * 3  # Upper bound: max 100 neurons per layer, up to 3 layers
    
    problem = {
        "fit_func": fitness_function,
        "bounds": (lb, ub),
        "ub": ub,
        "minmax": "min",  # Optimization direction (minimization)
        "log_to": None
    }
    
    # PSO phase
    pso = ps.single.GlobalBestPSO(n_particles=num_agents, dimensions=len(lb), options={'c1': 2.05, 'c2': 2.05, 'w': 0.729})
    best_pso_solution, best_objective_value = pso.optimize(fitness_function, iters=num_iterations)
    
    best_pso_solution = np.atleast_1d(best_pso_solution)  # Ensure it's an array
    return [int(round(neurons)) for neurons in best_pso_solution if neurons > 0], best_objective_value


def main_pipeline(input_df, X_columns, Y_column, alpha=0.5):
    """
    Main function to train and optimize an MLP model.
    """
    X = input_df[X_columns]
    y = input_df[[Y_column]]
    X_train, X_test, y_train, y_test = stratified_temperature_split(X, y, num_bins=10, test_size=0.2, random_state=123)
    
    best_layers, best_objective_value = optimize_mlp_architecture(X_train, y_train, X_test, y_test, alpha)
    final_mlp_model = MLP(activation='relu', hidden_layer_sizes=tuple(best_layers), learning_rate='constant',
                           max_iter=500, solver='adam', random_state=42)
    
    xnorm = MinMaxScaler().fit(X_train)
    X_train_norm = xnorm.transform(X_train)
    X_test_norm = xnorm.transform(X_test)
    
    ynorm = MinMaxScaler().fit(y_train)
    y_train_norm = ynorm.transform(y_train)
    
    final_mlp_model.fit(X_train_norm, y_train_norm)
    y_test_pred_norm = final_mlp_model.predict(X_test_norm).reshape(-1, 1)
    y_test_pred = ynorm.inverse_transform(y_test_pred_norm)
    
    return final_mlp_model, best_layers, best_objective_value, y_test.values.flatten().tolist(), y_test_pred.flatten().tolist()

# Example Usage:
# final_model, best_architecture, objective_value, y_true, y_pred = main_pipeline(dataframe, feature_columns, target_column, alpha=0.7)

# Example Usage:
final_model, best_architecture, objective_value, y_true, y_pred = main_pipeline(dataframe, feature_columns, target_column, alpha=0.7)
print(f'Best MLP Architecture found: {best_architecture}')

2025-03-25 10:44:51,227 - pyswarms.single.global_best - INFO - Optimize for 50 iters with {'c1': 2.05, 'c2': 2.05, 'w': 0.729}
pyswarms.single.global_best: 100%|██████████|50/50, best_cost=-0.533
2025-03-25 10:52:41,378 - pyswarms.single.global_best - INFO - Optimization finished | best cost: -0.5329583629331774, best pos: [29.3133675   0.68456235  1.5987726 ]
